## Classic Army Diet Optimization Problem

The diet problem was one of the first optimization problems studied in the 1930s and 1940s. The problem was motivated by the Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet.

Given a set of foods, along with the nutrient information for each food and the cost per serving of each food, the objective of the diet problem is to select the number of servings of each food to purchase (and consume) so as to minimize the cost of the food while meeting the specified nutritional requirements. Typically, the nutritional requirements are expressed as a minimum and a maximum allowable level for each nutritional component. Other constraints such a minimum and/or maximum number of servings may be included to improve the quality of the menu.

For more information, visit https://neos-guide.org/content/diet-problem

In [2]:
# Let's begin by importing PuLP modeler functions. We will also need the pandas library.

from pulp import *
import pandas as pd

In [3]:
#Create "prob" variable to store the problem data
prob=LpProblem("Diet Problem",LpMinimize)
M=999999999

In [17]:
#Next, we'll load our data into a pandas dataframe.
data=pd.read_excel("dietSummer2018.xls")
data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [18]:
#Looking at the data, the information on the types of foods reside in the first 64 rows of the data (index 0 to 63).
# Rows 65-67 contain information that help us define our constraints
data=data.loc[0:63,]

In [19]:
#Now, we will convert each row of data into a separate list (this is preprocessing so that we can apply the PuLP function later)
data=data.values.tolist()

In [20]:
# Next we will create a list of dictionary for each nutritional item, with foods as the key and amounts of serving
# (of the nutrition) as the value
foods=[x[0] for x in data]
priceperserving = dict([(x[0],float(x[1])) for x in data])
servingsize = dict([(x[0],x[2]) for x in data])
calories = dict([(x[0],float(x[3])) for x in data])
cholesterol = dict([(x[0],float(x[4])) for x in data])
totalfat = dict([(x[0],float(x[5])) for x in data])
sodium = dict([(x[0],float(x[6])) for x in data])
carbohydrates = dict([(x[0],float(x[7])) for x in data])
dietaryfiber = dict([(x[0],float(x[8])) for x in data])
protein = dict([(x[0],float(x[9])) for x in data])
vitA = dict([(x[0],float(x[10])) for x in data])
vitC = dict([(x[0],float(x[11])) for x in data])
calcium = dict([(x[0],float(x[12])) for x in data])
iron = dict([(x[0],float(x[13])) for x in data])

In [21]:
# Now we can define two dictionaries that contain the LP variables.
# food_vars stores variables defining how many servings of the food to eat. food_vars' key values are the food names, and
# its values are stored as Foods_FoodName. The LpVariables in food_vars have a lower bound of 0.
food_vars = LpVariable.dicts("Foods",foods,0)
# food_vars_bin stores variables defining whether the food was selected to be eaten. food_vars_bin' key values are the food
# names, and its values are stored as Chosen_FoodName. The LpVariables in food_vars_bin have a lower bound of 0 and upper
# bound of 1 (and has to be an integer).
food_vars_bin = LpVariable.dicts("Chosen",foods,0,1,'Integer')

In [22]:
# Next we will define our objective function for the optimization problem.
prob += lpSum([priceperserving[i]*food_vars[i]for i in foods])

In [23]:
# Based on the given minimum and maximum daily intakes (per nutrition), we've defined the constraints for our optimization
# problem
prob += lpSum([calories[i] * food_vars[i] for i in foods]) >= 1500
prob += lpSum([calories[i] * food_vars[i] for i in foods]) <= 2500
prob += (lpSum([cholesterol[i] * food_vars[i] for i in foods]) >= 30)
prob += (lpSum([cholesterol[i] * food_vars[i] for i in foods]) <= 240)
prob += (lpSum([totalfat[i] * food_vars[i] for i in foods]) >= 20)
prob += (lpSum([totalfat[i] * food_vars[i] for i in foods]) <= 70)
prob += (lpSum([sodium[i] * food_vars[i] for i in foods]) >= 800)
prob += (lpSum([sodium[i] * food_vars[i] for i in foods]) <= 2000)
prob += (lpSum([carbohydrates[i] * food_vars[i] for i in foods]) >= 130)
prob += (lpSum([carbohydrates[i] * food_vars[i] for i in foods]) <= 450)
prob += (lpSum([dietaryfiber[i] * food_vars[i] for i in foods]) >= 125)
prob += (lpSum([dietaryfiber[i] * food_vars[i] for i in foods]) <= 250)
prob += (lpSum([protein[i] * food_vars[i] for i in foods]) >= 60)
prob += (lpSum([protein[i] * food_vars[i] for i in foods]) <= 100)
prob += (lpSum([vitA[i] * food_vars[i] for i in foods]) >= 1000)
prob += (lpSum([vitA[i] * food_vars[i] for i in foods]) <= 10000)
prob += (lpSum([vitC[i] * food_vars[i] for i in foods]) >= 400)
prob += (lpSum([vitC[i] * food_vars[i] for i in foods]) <= 5000)
prob += (lpSum([calcium[i] * food_vars[i] for i in foods]) >= 700)
prob += (lpSum([calcium[i] * food_vars[i] for i in foods]) <=1500)
prob += (lpSum([iron[i] * food_vars[i] for i in foods]) >= 10)
prob += (lpSum([iron[i] * food_vars[i] for i in foods]) <= 40)

In [25]:
# Finally we can copy all the information defined above into an LP file and run the PuLP solver.
prob.writeLP("DietProblem.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

# Below are the results to our optimization problem
for v in prob.variables():
    print(v.name, "=", v.varValue)
# we will print the optimised objective function value
print("Total cost of the diet = ", value(prob.objective))

Status: Optimal
Foods_2%_Lowfat_Milk = 0.0
Foods_3.3%_Fat,Whole_Milk = 0.0
Foods_Apple,Raw,W_Skin = 0.0
Foods_Apple_Pie = 0.0
Foods_Bagels = 0.0
Foods_Banana = 0.0
Foods_Beanbacn_Soup,W_Watr = 0.0
Foods_Bologna,Turkey = 0.0
Foods_Butter,Regular = 0.0
Foods_Cap'N_Crunch = 0.0
Foods_Carrots,Raw = 0.0
Foods_Celery,_Raw = 52.64371
Foods_Cheddar_Cheese = 0.0
Foods_Cheerios = 0.0
Foods_Chicknoodl_Soup = 0.0
Foods_Chocolate_Chip_Cookies = 0.0
Foods_Corn_Flks,_Kellogg'S = 0.0
Foods_Couscous = 0.0
Foods_Crm_Mshrm_Soup,W_Mlk = 0.0
Foods_Frankfurter,_Beef = 0.0
Foods_Frozen_Broccoli = 0.25960653
Foods_Frozen_Corn = 0.0
Foods_Grapes = 0.0
Foods_Ham,Sliced,Extralean = 0.0
Foods_Hamburger_W_Toppings = 0.0
Foods_Hotdog,_Plain = 0.0
Foods_Kielbasa,Prk = 0.0
Foods_Kiwifruit,Raw,Fresh = 0.0
Foods_Lettuce,Iceberg,Raw = 63.988506
Foods_Macaroni,Ckd = 0.0
Foods_Malt_O_Meal,Choc = 0.0
Foods_New_E_Clamchwd,W_Mlk = 0.0
Foods_Neweng_Clamchwd = 0.0
Foods_Oatmeal = 0.0
Foods_Oatmeal_Cookies = 0.0
Foods_Oranges =

Shown above, the PuLP solver reveals that the cheapest diet that satisfies the maximum and minimum daily nutrition constraints consists of 52.6 servings of raw celery, 0.26 servings of frozen broccoli, 63.98 servings of raw iceberg lettuce, 2.29 servings of oranges, 0.14 servings of poached eggs, and 13.87 servings of air-popped popcorn. The total cost of this diet is $4.34.

What if, however, we want to add a constraint that if a food has been selected, then a minimum of 1/10 serving must be chosen? Recall that we've already previously defined "food_vars_bin", which stores a binary variable defining whether or not a food has been selected. We will use such variable to define the new constraint to our problem.

In [28]:
for i in foods:
    prob += food_vars[i]<=M*food_vars_bin[i]
    prob += food_vars[i]>=0.1*food_vars_bin[i]

# We can now re-run the solver and observe if there are any changes to the results.
prob.writeLP("DietProblem.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
# we will print the optimised objective function value
print("Total cost of the diet = ", value(prob.objective))

Status: Optimal
Chosen_2%_Lowfat_Milk = 0.0
Chosen_3.3%_Fat,Whole_Milk = 0.0
Chosen_Apple,Raw,W_Skin = 0.0
Chosen_Apple_Pie = 0.0
Chosen_Bagels = 0.0
Chosen_Banana = 0.0
Chosen_Beanbacn_Soup,W_Watr = 0.0
Chosen_Bologna,Turkey = 0.0
Chosen_Butter,Regular = 0.0
Chosen_Cap'N_Crunch = 0.0
Chosen_Carrots,Raw = 0.0
Chosen_Celery,_Raw = 1.0
Chosen_Cheddar_Cheese = 0.0
Chosen_Cheerios = 0.0
Chosen_Chicknoodl_Soup = 0.0
Chosen_Chocolate_Chip_Cookies = 0.0
Chosen_Corn_Flks,_Kellogg'S = 0.0
Chosen_Couscous = 0.0
Chosen_Crm_Mshrm_Soup,W_Mlk = 0.0
Chosen_Frankfurter,_Beef = 0.0
Chosen_Frozen_Broccoli = 1.0
Chosen_Frozen_Corn = 0.0
Chosen_Grapes = 0.0
Chosen_Ham,Sliced,Extralean = 0.0
Chosen_Hamburger_W_Toppings = 0.0
Chosen_Hotdog,_Plain = 0.0
Chosen_Kielbasa,Prk = 0.0
Chosen_Kiwifruit,Raw,Fresh = 0.0
Chosen_Lettuce,Iceberg,Raw = 1.0
Chosen_Macaroni,Ckd = 0.0
Chosen_Malt_O_Meal,Choc = 0.0
Chosen_New_E_Clamchwd,W_Mlk = 0.0
Chosen_Neweng_Clamchwd = 0.0
Chosen_Oatmeal = 0.0
Chosen_Oatmeal_Cookies = 0.

As shown above, the newly added constraint (that if a food has been selected, then a minimum of 1/10 serving must be chosen) did not impact our optimization results from 15.2.1. This is due to the fact that all the suggested food servings by the model was larger than 1/10 serving, and so the constraint never truly took effect. 

As we add more constraints below, however, we will see this constraint in effect (in the final solution).

In [29]:
#Let's add one more constraint: the model cannot select both celery and frozen broccoli at once (it just doesn't taste too great together).

prob += (food_vars_bin['Frozen Broccoli']+food_vars_bin["Celery, Raw"])<=1

In [30]:
# We can now re-run the solver and observe if there are any changes to the results.
prob.writeLP("DietProblem.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)
# we will print the optimised objective function value
print("Total cost of the diet = ", value(prob.objective))


Status: Optimal
Chosen_2%_Lowfat_Milk = 0.0
Chosen_3.3%_Fat,Whole_Milk = 0.0
Chosen_Apple,Raw,W_Skin = 0.0
Chosen_Apple_Pie = 0.0
Chosen_Bagels = 0.0
Chosen_Banana = 0.0
Chosen_Beanbacn_Soup,W_Watr = 0.0
Chosen_Bologna,Turkey = 0.0
Chosen_Butter,Regular = 0.0
Chosen_Cap'N_Crunch = 0.0
Chosen_Carrots,Raw = 0.0
Chosen_Celery,_Raw = 1.0
Chosen_Cheddar_Cheese = 0.0
Chosen_Cheerios = 0.0
Chosen_Chicknoodl_Soup = 0.0
Chosen_Chocolate_Chip_Cookies = 0.0
Chosen_Corn_Flks,_Kellogg'S = 0.0
Chosen_Couscous = 0.0
Chosen_Crm_Mshrm_Soup,W_Mlk = 0.0
Chosen_Frankfurter,_Beef = 0.0
Chosen_Frozen_Broccoli = 0.0
Chosen_Frozen_Corn = 0.0
Chosen_Grapes = 0.0
Chosen_Ham,Sliced,Extralean = 0.0
Chosen_Hamburger_W_Toppings = 0.0
Chosen_Hotdog,_Plain = 0.0
Chosen_Kielbasa,Prk = 0.0
Chosen_Kiwifruit,Raw,Fresh = 0.0
Chosen_Lettuce,Iceberg,Raw = 1.0
Chosen_Macaroni,Ckd = 0.0
Chosen_Malt_O_Meal,Choc = 0.0
Chosen_New_E_Clamchwd,W_Mlk = 0.0
Chosen_Neweng_Clamchwd = 0.0
Chosen_Oatmeal = 0.0
Chosen_Oatmeal_Cookies = 0.

We can see that the solver took the new constraint into account; previously our optimal diet consists of both servings of raw celery and frozen broccoli. Now, our model selected only raw celery in the diet (and excluded broccoli).
Shown above, our new optimal diet (with the new constraint) consists of 42.40 servings of raw celery, 82.80 servings of raw iceberg lettuce, 3.08 servings of oranges, 1.94 servings of peanut butter, 0.1 servings of poached eggs, 13.22 servings of air-popped popcorn and 0.1 servings of scrambled eggs. The total cost of this new diet is $4.49. Intuitively, it makes sense that this new diet costs more than the previous given that we've added a new constraint to essentially the same problem.

Finally, let's see how our model can be altered to handle a few more constraints to this LP optimization problem.

In [34]:
# Let's define another new constraint so that the model must select at least 3 types of proteins. For the purpose of this analysis, I've defined the following to be proteins:
# 'Roasted Chicken', 'Poached Eggs', 'Scrambled Eggs', 'Bologna,Turkey', 'Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', 'Pizza W/Pepperoni', 'Taco',
# 'Hamburger W/Toppings', 'Hotdog, Plain', 'Pork', 'Sardines in Oil', 'White Tuna in Water

prob += (food_vars_bin['Roasted Chicken']+food_vars_bin['Poached Eggs']+food_vars_bin['Scrambled Eggs']
    +food_vars_bin['Bologna,Turkey']+food_vars_bin['Frankfurter, Beef']+food_vars_bin['Ham,Sliced,Extralean']
    +food_vars_bin['Kielbasa,Prk']+food_vars_bin['Pizza W/Pepperoni']+food_vars_bin['Taco']
    +food_vars_bin['Hamburger W/Toppings']+food_vars_bin['Hotdog, Plain']+food_vars_bin['Pork']
    +food_vars_bin['Sardines in Oil']+food_vars_bin['White Tuna in Water']
    )>=3.0

In [36]:
# We can now re-run the solver and observe if there are any changes to the results.
prob.writeLP("DietProblem.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
# we will print the optimized objective function value
print("Total cost of the diet = ", value(prob.objective))

Status: Optimal
Chosen_2%_Lowfat_Milk = 0.0
Chosen_3.3%_Fat,Whole_Milk = 0.0
Chosen_Apple,Raw,W_Skin = 0.0
Chosen_Apple_Pie = 0.0
Chosen_Bagels = 0.0
Chosen_Banana = 0.0
Chosen_Beanbacn_Soup,W_Watr = 0.0
Chosen_Bologna,Turkey = 0.0
Chosen_Butter,Regular = 0.0
Chosen_Cap'N_Crunch = 0.0
Chosen_Carrots,Raw = 0.0
Chosen_Celery,_Raw = 1.0
Chosen_Cheddar_Cheese = 0.0
Chosen_Cheerios = 0.0
Chosen_Chicknoodl_Soup = 0.0
Chosen_Chocolate_Chip_Cookies = 0.0
Chosen_Corn_Flks,_Kellogg'S = 0.0
Chosen_Couscous = 0.0
Chosen_Crm_Mshrm_Soup,W_Mlk = 0.0
Chosen_Frankfurter,_Beef = 0.0
Chosen_Frozen_Broccoli = 0.0
Chosen_Frozen_Corn = 0.0
Chosen_Grapes = 0.0
Chosen_Ham,Sliced,Extralean = 0.0
Chosen_Hamburger_W_Toppings = 0.0
Chosen_Hotdog,_Plain = 0.0
Chosen_Kielbasa,Prk = 1.0
Chosen_Kiwifruit,Raw,Fresh = 0.0
Chosen_Lettuce,Iceberg,Raw = 1.0
Chosen_Macaroni,Ckd = 0.0
Chosen_Malt_O_Meal,Choc = 0.0
Chosen_New_E_Clamchwd,W_Mlk = 0.0
Chosen_Neweng_Clamchwd = 0.0
Chosen_Oatmeal = 0.0
Chosen_Oatmeal_Cookies = 0.

Once again, it is evident that the solver took the new constraint (of having at least 3 types of proteins) into account. In this new optimal diet, we can see that scrambled eggs,poached eggs,and kielbasa were selected.
Our final optimal diet (with the new constraints added from the entire Question 15.2.2) is as follows: 42.40 servings of raw celery, 0.1 servings of kielbasa, 82.80 servings of raw iceberg lettuce, 3.07 servings of oranges, 1.94 servings of peanut butter, 0.1 servings of poached eggs, 13.22 servings of air-popped popcorn and 0.1 servings of scrambled eggs. The total cost of this diet is $4.51.